Assignment 21

In [77]:
import numpy as np

class Player:
    def __init__(self, id, pos, score = 0):
        self.id = id
        self.score = score
        self.position = pos
        self.universesWon = np.uint64(0)

def PlayTillEndDeterministic(players):
    diceRoll = 0
    
    index = 0
    while(True):
        player = players[index]
        for k in range (3):
            diceValue = 0
            diceRoll +=1
            if diceRoll % 100 == 0:
                diceValue = 100
            else:
                diceValue = diceRoll % 100
            player.position += diceValue 
        if player.position %10 == 0:
                player.score += 10
                player.position = 10
        else:
            player.position = player.position % 10
            player.score += player.position
        if (player.score >= 1000):
            break
        index = (index + 1)%len(players)
    return players, diceRoll
        
    


In [78]:
players, rollTimes = PlayTillEndDeterministic([Player(1, 4), Player(2, 8)])
player = min(players, key=lambda x: x.score)

print(player.score * rollTimes)

Player with lowest score: <__main__.Player object at 0x0000025DD5714D00>
739785


In [79]:
players, rollTimes = PlayTillEndDeterministic([Player(0, 6), Player(1, 1)])
player = min(players, key=lambda x: x.score)

print(player.score * rollTimes)

Player with lowest score: <__main__.Player object at 0x0000025DD5717D00>
929625


In [96]:
cash = []
           # [3,4,5,6,7,8,9]
probablity = [1,3,6,7,6,3,1]

In [97]:
def PrePareNextRoll(i,player, antiplayer, depth, step):
    tmp = [Player(1, i, player.score), Player(2, antiplayer.position, antiplayer.score)]
    tmp[0].score += i
    DiracDice(tmp[1], tmp[0], depth)
    player.universesWon += tmp[0].universesWon * np.uint64(probablity[step])
    antiplayer.universesWon += tmp[1].universesWon * np.uint64(probablity[step])

def DiracDice(player, antiplayer, depth = 0):
    depth += 1
    
    cashfind = None
    if len(cash) > 0:
        cashfind = [x for x in cash if x[0] == player.score and x[1] == player.position and x[2] == antiplayer.score and x[3] == antiplayer.position]
    if cashfind is not None and len(cashfind) > 0:
        #print("found cash: {}".format(cashfind))
        player.universesWon += cashfind[0][4][0]
        antiplayer.universesWon += cashfind[0][4][1]
    else:
        for i in range(3,10):
            value = (player.position + i)%10
            if value == 0:
                value = 10
            if (player.score + value>=21):
                player.universesWon += np.uint64(probablity[i-3])
            else:
                PrePareNextRoll( value, player, antiplayer, depth, i-3)

        cash.append([player.score, player.position, antiplayer.score, antiplayer.position,  [player.universesWon, antiplayer.universesWon]])
        #print("cash: {}".format(cash[-1]))    

In [98]:
player1= Player(1, 4)
player2= Player(2, 8)
DiracDice(player1, player2)
print(player1.universesWon)
print(player2.universesWon)

444356092776315
341960390180808


In [99]:
player1 = Player(1, 6)
player2= Player(2, 1)
DiracDice(player1, player2)
print(player1.universesWon)
print(player2.universesWon)

153087536629019
175731756652760
